# Exploratory Data Analysis & Interactive KPI Dashboard

**Objective:**  
This notebook delivers a comprehensive exploratory analysis of our master dataset—loaded from GCS as Parquet—covering data quality, schema drift, category- and product-level performance, pricing & sentiment dynamics, temporal trends, and review-level insights. It’s designed to surface actionable intelligence and empower you to tweak parameters on the fly to uncover deeper patterns.

---

## Process Walkthrough

1. **Environment Setup & Data Ingestion**  
   - Import core Python, PySpark, Bokeh and widget libraries  
   - Connect to an existing SparkSession (or create one)  
   - Read the master dataset from `gs://ba843-group1-project/master_data.parquet` into a Spark DataFrame  

2. **Data Quality Assessment**  
   - Define a helper (`null_or_empty`) to flag missing or empty values  
   - Compute the percentage of null/empty entries per column  
   - Visualize completeness using a Bokeh bar chart, highlighting any quality gaps  

3. **Schema Drift Watch**  
   - Aggregate schema metrics over time (e.g., column counts, data-type changes)  
   - Plot drift signals (e.g., sudden new columns or shifting distributions)  
   - Quickly identify structural changes that may impact downstream analyses  

4. **Category KPI Scorecard**  
   - Calculate key metrics (e.g., average rating, review count, helpfulness ratio) for each `main_category`  
   - Render an interactive table & bar chart so you can sort, filter or highlight categories  

5. **Star-Mix Matrix**  
   - Build a cross-tab of star ratings by category to spot rating concentration patterns  
   - Use color-coded cells to flag over- or under-represented rating buckets  

6. **Best/Worst SKU Leaderboard**  
   - Assemble SKU-level statistics (e.g., average rating, review volume, helpfulness)  
   - Expose four widgets—**Category**, **Metric**, **Best/Worst**, **Top N**—to slice and sort your leaderboard  
   - Rapidly surface top or bottom performers in any category or metric  

7. **Price-vs-Rating Sweet-Spot**  
   - Overlay price bands against average rating to locate the “sweet-spot” where value meets satisfaction  
   - Allow adjustable price ranges and rating thresholds for on-the-fly exploration  

8. **Price Elasticity Bands**  
   - Segment items into price-elastic bands and measure demand sensitivity  
   - Interactive sliders let you redefine price intervals and instantly view elasticity shifts  

9. **Monthly Sentiment Pulse**  
   - Compute and plot average sentiment scores by month to track user mood over time  
   - Drill into specific months to uncover seasonality or campaign effects  

10. **Seasonality Heatmap**  
    - Heatmap of review volume or sentiment by weekday & hour to expose peak engagement windows  
    - Time-range picker lets you focus on custom date intervals  

11. **Delight Themes Radar**  
    - Perform topic modeling or keyword extraction on “delight” comments  
    - Render a radar chart of emerging themes; adjust topic count via a widget  

12. **Sentiment Imbalance Gauge**  
    - Gauge chart contrasting positive vs. negative mentions for a quick health check  
    - Tweak sentiment thresholds interactively to recalibrate your gauge  

13. **Review Velocity Spike Alert**  
    - Detect and highlight sudden bursts in review volume (potential viral events or issues)  
    - Use a date-range selector and spike-sensitivity slider to fine-tune alerts  

14. **Helpfulness vs. Rating**  
    - Hexbin scatter with marginal distributions showing density zones for helpfulness vs. rating  
    - Hover tooltips reveal precise counts and densities  

15. **Verified-Purchase Impact**  
    - Interactive dumbbell chart comparing average ratings for verified vs. unverified purchases  
    - Click or hover to see Δ-star differences for each category or SKU  

---

## Interactive Exploration & Parameterization

Throughout the notebook, ipywidgets sliders, dropdowns and toggles are wired to Bokeh callbacks—so you can:

- **Switch categories** and **metrics**  
- **Define Top N** thresholds for leaderboards  
- **Adjust price ranges**, **time windows**, **sentiment cuts** and more  
- **Instantly refresh** plots and tables without re-running the entire notebook  

This empowers you to experiment with hypotheses, zero in on key segments, and derive tailored insights—turning a static analysis into a living, decision-driven dashboard.  


## 1. Environment Setup & Data Ingestion

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# 1) Core Libraries
# ─────────────────────────────────────────────────────────────────────────────
import numpy as np
import pandas as pd

# ─────────────────────────────────────────────────────────────────────────────
# 2) PySpark
# ─────────────────────────────────────────────────────────────────────────────
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import (
    col, when, size, isnan, lit,
    count as spark_count, sum as spark_sum,
    avg, countDistinct, min as spark_min,
    from_unixtime, date_trunc, year, month,
    floor, expr, lower, length, explode, count,
    length as str_length, date_trunc
)

from pyspark.sql.functions import expr
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lower, length
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
from pyspark.sql.functions import explode


from pyspark.sql.types import (
    StringType, ArrayType, MapType,
    StructType, DoubleType, FloatType
)

# ─────────────────────────────────────────────────────────────────────────────
# 3) PySpark ML (NLP)
# ─────────────────────────────────────────────────────────────────────────────
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover

# ─────────────────────────────────────────────────────────────────────────────
# 4) Plotting
# ─────────────────────────────────────────────────────────────────────────────
import plotly.express as px

from bokeh.resources import INLINE
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import gridplot

# 3️⃣ Bokeh setup
from bokeh.plotting   import figure, output_notebook, show
from bokeh.models     import ColumnDataSource, HoverTool, LinearColorMapper, ColorBar, Span
from bokeh.palettes   import Viridis256
from bokeh.util.hex   import hexbin


from bokeh.models import (
    ColumnDataSource, HoverTool, LinearColorMapper,
    ColorBar, Span, Band, Range1d, LinearAxis,
    DatetimeTickFormatter
)
from bokeh.palettes import Viridis256, Category10
from bokeh.util.hex import hexbin

# ─────────────────────────────────────────────────────────────────────────────
# 5) Notebook UI
# ─────────────────────────────────────────────────────────────────────────────
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# ─────────────────────────────────────────────────────────────────────────────
# Initialize Bokeh in Jupyter
# ─────────────────────────────────────────────────────────────────────────────
output_notebook()


Loading BokehJS ...

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:474)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)


In [ ]:
# Re-use an existing SparkSession if you already have one
spark = SparkSession.builder.getOrCreate()

In [ ]:
path = "gs://ba843-group1-project/master_data.parquet"

In [ ]:
# Fresh read (edit path if needed)
master_df = spark.read.parquet(path)         # cache for repeated scans
master_df.cache()

DataFrame[parent_asin: string, asin: string, helpful_vote: bigint, images: array<struct<attachment_type:string,large_image_url:string,medium_image_url:string,small_image_url:string>>, rating: double, text: string, timestamp: bigint, title: string, user_id: string, verified_purchase: boolean, main_category: string, average_rating: double, rating_number: double, features: string, description: string, price: double, videos: string, store: string, categories: string, bought_together: string, Product_title: string]

## 2. Data Quality Assessment

In [ ]:
def null_or_empty(col_name, dtype):
    """
    Returns a Column expression that is TRUE when the value is
    null  OR empty (type-specific).
    """
    c = col(col_name)

    # Always count plain null first
    cond = c.isNull()

    if isinstance(dtype, StringType):
        cond = cond | (c == "")                       # empty string
    elif isinstance(dtype, (DoubleType, FloatType)):
        cond = cond | isnan(c)                        # NaN = empty for numerics
    elif isinstance(dtype, ArrayType):
        cond = cond | (size(c) == 0)                  # empty list
    elif isinstance(dtype, (MapType, StructType)):
        # Spark has no native “isEmpty” for structs/maps; just keep null check
        pass

    return cond

In [ ]:
# 2) Compute null/empty % for each column in master_df
exprs = []
for c, t in master_df.dtypes:
    cond = col(c).isNull()
    if t == "string":
        cond = cond | (col(c) == "")
    elif t.startswith("array"):
        cond = cond | (size(col(c)) == 0)
    elif t in ("double", "float"):
        cond = cond | isnan(col(c))
    exprs.append(
        (spark_sum(when(cond, 1).otherwise(0)) / spark_sum(lit(1)) * 100)
        .alias(c)
    )

overall_nulls = (
    master_df
      .select(exprs)
      .toPandas()
      .T
      .reset_index()
      .rename(columns={"index": "column", 0: "null_pct"})
      .sort_values("null_pct", ascending=False)
      .reset_index(drop=True)
)

In [ ]:
# 3) Prep Bokeh source with colors
df = overall_nulls.copy()
palette = Category10[10] * ((len(df)//10) + 1)
df["color"] = palette[:len(df)]
source = ColumnDataSource(df)

# 4) Build interactive horizontal bar chart
p = figure(
    y_range=df["column"].tolist(),
    height=600,
    width=800,
    title="🔍 Data-Health Check: Null / Empty % by Column",
    toolbar_location="above",
    tools="pan,wheel_zoom,box_zoom,reset,hover"
)
p.hbar(
    y="column",
    right="null_pct",
    height=0.8,
    color="color",
    source=source
)

# 5) Tweak axes & hover
p.x_range.start = 0
p.xaxis.axis_label = "Null / Empty %"
p.ygrid.grid_line_color = None

hover = p.select_one(HoverTool)
hover.tooltips = [
    ("Column", "@column"),
    ("Null %", "@null_pct{0.0}%")
]

# 6) Show it
show(p)

# Data Quality Assesment Visualizations Image

## Null Percentage
![Null Percentage](https://github.com/billburr958/images-temp/blob/main/Exploration/null_percent.png?raw=true)


- We noticed that the **bought_together** column is entirely empty (100% missing), so we’ll not use it since it can’t contribute to our analysis.  
- It is observed that **price** is missing for roughly one-third of records (≈33%).
- All of the core review fields—**rating**, **helpful_vote**, **verified_purchase**, etc.—are fully populated, which gives us confidence to proceed with leaderboards, trend analyses, and KPI calculations without worrying about gaps.  

---

## 3. Schema Drift Watch

In [ ]:
# Extract week from the millisecond timestamp & compute cardinalities

# Convert the millisecond timestamp to a Spark timestamp, then truncate to week
weekly_card_spark = (
    master_df
      .withColumn(
          "week",
          date_trunc(
              "week",
              from_unixtime(col("timestamp")/1000).cast("timestamp")
          )
      )
      .groupBy("week")
      .agg(
          *[countDistinct(c).alias(c) for c in master_df.columns]
      )
      .orderBy("week")
)

In [ ]:
# 2.1 List all metric columns (everything except “week”)
metrics = [c for c in weekly_card_spark.columns if c != "week"]
n_metrics = len(metrics)

# 2.2 Use stack() to unpivot: produces (week, column, cardinality)
exprs = ", ".join(f"'{c}', {c}" for c in metrics)
weekly_long = weekly_card_spark.selectExpr(
    "week",
    f"stack({n_metrics}, {exprs}) as (column, cardinality)"
)

# 2.3 Rank by cardinality within each week (1 = highest cardinality)
w = Window.partitionBy("week").orderBy(expr("cardinality").desc())
weekly_ranked = weekly_long.withColumn("rank", rank().over(w))

In [ ]:
# 3️⃣ Convert to Pandas & prepare
weekly_ranked_pd = (
    weekly_ranked
      .toPandas()
)

# Ensure 'week' is a datetime and sort
weekly_ranked_pd['week'] = pd.to_datetime(weekly_ranked_pd['week'])
weekly_ranked_pd = weekly_ranked_pd.sort_values(['week', 'rank'])

In [ ]:
weeks = sorted(weekly_ranked_pd['week'].unique())

def plot_for_week(idx):
    week = weeks[idx]
    dfw = (
        weekly_ranked_pd
        [weekly_ranked_pd['week'] == week]
        .sort_values('rank')
    )
    source = ColumnDataSource(dfw)

    p = figure(
        y_range=list(reversed(dfw['column'].tolist())),
        height=500, width=700,
        title=f"Schema Drift on {week.date()}",
        tools="hover,reset"
    )
    p.hbar(
        y='column',
        right='rank',
        height=0.6,
        source=source,
        color='teal'
    )
    p.xaxis.axis_label = "Rank (1 = highest cardinality)"
    p.x_range.start = 1
    p.x_range.end   = len(metrics)

    hover = p.select_one(HoverTool)
    hover.tooltips = [("Column","@column"),("Rank","@rank")]

    show(p)

# Slider widget to pick week index
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(weeks)-1,
    step=1,
    description='Week #',
    continuous_update=False
)

out = widgets.Output()

def on_slide(change):
    with out:
        clear_output(wait=True)
        plot_for_week(change['new'])

slider.observe(on_slide, names='value')

# Display
display(widgets.VBox([slider, out]))

# Initial plot
plot_for_week(0)

## Schema Drift Dashboard
![Schema Drift Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/schema_drift.png?raw=true)

- **Stable, High-Growth Core Keys**  
  - **`parent_asin`, `asin`, `text`, `timestamp`, `Product_title`** each grew their distinct counts consistently over the period.  
  - Their **low rank volatility** (std_rank ≈ 0.4–1.6) shows they reliably lead the schema every week—ideal anchors for joins, lookups, and trend analyses.

- **Rapidly Expanding Metadata**  
  - **`price`** (+77), **`videos`** (+79), **`store`** (+122) and **`categories`** (+102) saw significant cardinality increases as more listings gained these attributes.  
  - Moderate rank swings (std_rank ≈ 3.2–3.9) indicate these fields’ relative importance can fluctuate—worth monitoring if downstream workflows depend on them.

- **High Volatility, Low-Growth Fields**  
  - **`rating`** (+4), **`average_rating`** (+15), **`helpful_vote`** (+1) and **`verified_purchase`** (+1) show minimal growth but **large rank standard deviations** (std_rank ≈ 5.0–5.3).  
  - These spikes suggest intermittent schema or ingestion hiccups; set up alerts for unexpected rank jumps to catch missing or malformed review data.

- **Deprecated Column**  
  - **`bought_together`** remains at **0 distinct values**, confirming it’s unused and can be safely removed or archived.

----

## 4. Category KPI Scorecard

In [ ]:
# 1) Aggregate KPIs per category
kpi_df = (
    master_df
      .groupBy("main_category")
      .agg(
          avg(col("rating")).alias("avg_rating"),                                    # ★☆☆ average score
          count(lit(1)).alias("n_reviews"),                                           # total reviews
          (spark_sum(when(col("rating") == 5, 1).otherwise(0))
            / count(lit(1)) * 100
          ).alias("pct_5_star"),                                                       # % 5-star
          expr("percentile_approx(price, 0.5)").alias("median_price"),                # median price
          (spark_sum(col("helpful_vote"))
            / (count(lit(1)) + lit(1))
          ).alias("helpfulness_ratio"),                                                # helpful-vote ratio
          (spark_sum(when(col("verified_purchase") == True, 1).otherwise(0))
            / count(lit(1)) * 100
          ).alias("pct_verified")                                                      # % verified
      )
)

# 2) Preview
kpi_df.show(2, truncate=False)

In [ ]:
# 3️⃣ Choose your category and convert to Pandas
# from pyspark.sql.functions import col


# Filter Spark DF and collect into a single-row Pandas DataFrame
kpi_pd = (
    kpi_df
      .select(
          "avg_rating",
          "n_reviews",
          "pct_5_star",
          "median_price",
          "helpfulness_ratio",
          "pct_verified",
          "main_category"
      )
      .toPandas()
)

# Reorder columns
kpi_pd = kpi_pd[
    ["main_category", "avg_rating", "n_reviews", "pct_5_star",
     "median_price", "helpfulness_ratio", "pct_verified"]
]

In [ ]:
# 2) Gather all categories for the dropdown
raw = [row["main_category"] for row in kpi_df.select("main_category").distinct().collect()]
categories = [c for c in raw if c]      # drop None or empty strings
categories.sort()
# 3) Base target defaults
targets = {
    "Average Rating":    4.2,
    "Reviews": 10000,
    "% 5-Star":          50,
    "Median Price ($)":  45,
    "Helpfulness Ratio": 0.20,
    "% Verified":        80,
}
col_map = {
    "Average Rating":    "avg_rating",
    "Reviews": "n_reviews",
    "% 5-Star":          "pct_5_star",
    "Median Price ($)":  "median_price",
    "Helpfulness Ratio": "helpfulness_ratio",
    "% Verified":        "pct_verified",
}

# 4) Build widgets
category_dd = widgets.Dropdown(options=categories, description="Category:")
threshold_wids = []
for label, default in targets.items():
    if isinstance(default, int):
        w = widgets.IntText(value=default, description=label)
    else:
        w = widgets.FloatText(value=default, description=label)
    threshold_wids.append(w)

controls = widgets.VBox([category_dd] + threshold_wids)
out = widgets.Output()

def update_chart(_):
    with out:
        clear_output(wait=True)
        # a) read current settings
        cat = category_dd.value
        curr_targs = {w.description: w.value for w in threshold_wids}

        # b) pull KPIs for this category
        pdf = (
            kpi_df
              .filter(col("main_category") == cat)
              .select(*col_map.values())
              .toPandas()
        )
        if pdf.empty:
            print(f"No data for category {cat}")
            return
        row = pdf.iloc[0]

        # c) build performance table
        rows = []
        for label, targ in curr_targs.items():
            actual = float(row[col_map[label]])
            pct = (actual / targ * 100) if targ else 0
            rows.append({"metric": label, "actual": actual, "target": targ, "pct": pct})
        perf = pd.DataFrame(rows).sort_values("pct").reset_index(drop=True)
        perf["norm"] = (perf["pct"] / 100).clip(upper=1.0)

        # d) radar chart
        fig = px.line_polar(
            perf, r="norm", theta="metric",
            line_close=True, markers=True,
            title=f"KPI Radar: {cat}",
            template="plotly_dark",
            color_discrete_sequence=["#2ecc71"]
        )
        fig.update_traces(
            fill="toself",
            hovertemplate=(
                "<b>%{theta}</b><br>"
                "Actual: %{customdata[0]:.2f}<br>"
                "Target: %{customdata[1]:.2f}<br>"
                "Perf: %{r:.0%}<extra></extra>"
            ),
            customdata=perf[["actual","target"]].values
        )
        fig.update_polars(
            radialaxis=dict(range=[0,1], tickvals=[0,0.5,1], ticktext=["0%","50%","100%"]),
            angularaxis=dict(direction="clockwise")
        )

        display(HTML(fig.to_html(include_plotlyjs="cdn", full_html=False)))

# 5) Wire up listeners
category_dd.observe(update_chart, names="value")
for w in threshold_wids:
    w.observe(update_chart, names="value")

# 6) Show controls + initial chart
display(controls, out)
update_chart(None)

## Category KPI Scorecard Dashboard
![Category KPI Scorecard Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/kpi_scorecard.png?raw=true)

# Category KPI Scorecard Insights & Recommendations

Below is a structured overview of each category’s performance—highlighting strengths, weaknesses, and actionable next steps.

| Category            | Avg. Rating | # Reviews | % 5★ | Median Price | Helpfulness Ratio | % Verified |
|---------------------|------------:|----------:|-----:|-------------:|------------------:|-----------:|
| Computers           |       4.36  |    40,471 | 70.2 |       \$34.95|             87.3% |      94.4% |
| All Electronics     |       4.10  |    22,625 | 61.9 |       \$16.98|             90.0% |      91.8% |
| **Overall (ALL)**   |       4.18  | 2,740,763 | 64.4 |       \$27.99|             86.2% |      92.9% |
| Grocery             |       3.91  |     4,887 | 59.9 |        \$8.07|             79.9% |      94.9% |
| Books               |       4.14  |    84,352 | 60.4 |       \$14.97|             62.7% |      95.0% |

## Key Observations

1. **Top-Rated & High-Volume Leaders**  
   - **Computers** boasts the highest average rating (4.36) and % 5★ (70.2%), with strong review engagement (87.3% helpfulness).  
   - Its median price (\$34.95) positions it as a mid-ticket category—strong unit economics and satisfied customers.

2. **Engagement Hotspots**  
   - **All Electronics** has the highest helpfulness ratio (90.0%)—indicating highly engaged reviewers, despite a slightly lower average rating (4.10).  
   - This suggests an opportunity to upsell accessories or bundle promotions to an active, opinionated audience.

3. **Categories Under Pressure**  
   - **Grocery** lags on rating (3.91) and helpfulness (79.9%)—its low median price (\$8.07) means small margins; improving product quality or clearer descriptions could boost confidence.  
   - **Books** shows only 62.7% helpfulness (lowest), despite a robust review count (84K). Consider incentives for detailed reviews or better topic segmentation to foster richer feedback.

4. **Verified Purchase Trust**  
   - All categories exceed 90% verified-purchase rates, with **Books** and **Grocery** highest (>94%). This high authenticity foundation can support loyalty campaigns or subscription models.

## Recommendations

- **Double Down on “Computers”**  
  - Expand high-margin peripherals and software bundles—leverage strong satisfaction to boost AOV.  
  - Feature top-rated SKUs prominently and encourage video demos to capitalize on engaged customers.

- **Activate Electronics Community**  
  - Gamify reviews (badges, leaderboards) given the high helpfulness—drive further content creation and social proof.  
  - Test cross-selling warranty and service plans as add-ons to increase lifetime value.

- **Revitalize Grocery**  
  - Audit lowest-rated subcategories; refine product images and descriptions to set clearer expectations.  
  - Introduce “taste test” video snippets or customer stories to enhance perceived quality.

- **Elevate Book Engagement**  
  - Implement targeted review campaigns (e.g., “Review 3 books, get 1 free”) to boost helpfulness ratio.  
  - Curate thematic collections (“Staff Picks”, “Award Winners”) and solicit expert blurbs to enrich content.

- **Monitor & Iterate**  
  - Track these KPIs monthly; flag any ±5% swings in rating or helpfulness for immediate root-cause analysis.  
  - Use segment-specific dashboards to A/B test descriptions, pricing, and review incentives—optimizing continuously based on data-driven feedback.


-----

## 5. Star-Mix Matrix

In [ ]:
master_enriched = master_df.withColumn(
    "review_length",
    str_length(col("text"))
)

In [ ]:
# 1.1) Extract all non-null main_category values
categories = (
    master_df
      .select("main_category")
      .where(col("main_category").isNotNull())
      .distinct()
      .rdd
      .map(lambda row: row["main_category"])
      .collect()
)
categories.sort()

In [ ]:
categories = sorted([c for c in categories if c])
category_dd = widgets.Dropdown(options=categories, description="Category:")

# 2️⃣ Build product dropdown (initially empty)
product_dd = widgets.Dropdown(options=[], description="Product:",
                              layout=widgets.Layout(width="500px"))

# 3️⃣ Update product list when category changes
def update_products(change):
    cat = category_dd.value
    # pull distinct product titles for this category
    prods = (
        master_df
          .filter(col("main_category") == cat)
          .select("Product_title")
          .where(col("Product_title").isNotNull())
          .distinct()
          .rdd.map(lambda r: r["Product_title"])
          .collect()
    )
    prods = sorted(prods)
    # Optionally limit to top 100 for performance
    product_dd.options = ["ALL"] + prods[:100]

category_dd.observe(update_products, names="value")

# 4️⃣ Output area for the plot
out = widgets.Output()

# 5️⃣ Define the update function
def update_plot(_):
    with out:
        clear_output(wait=True)
        cat  = category_dd.value
        prod = product_dd.value
        bin_width = 20

        # Spark-side histogram
        sdf = master_df.withColumn("review_length", str_length(col("text")))
        sdf = sdf.filter(col("main_category") == cat)
        if prod != "ALL":
            sdf = sdf.filter(col("Product_title") == prod)

        hist_spark = (
            sdf
              .withColumn("length_bin", floor(col("review_length")/bin_width)*bin_width)
              .groupBy("rating", "length_bin")
              .count()
              .orderBy("rating", "length_bin")
        )

        # To Pandas + labels + density
        hist_pd = (
            hist_spark
              .withColumnRenamed("count","cnt")
              .toPandas()
        )
        hist_pd["rating_int"] = hist_pd["rating"].astype(int)
        hist_pd["rating_str"] = hist_pd["rating_int"].astype(str) + "★"
        hist_pd["density"]   = (
            hist_pd.groupby("rating_str")["cnt"]
                   .transform(lambda x: x/x.sum())
        )

        # Prepare offsets
        rating_strs = sorted(hist_pd["rating_str"].unique(), reverse=True)
        max_d = hist_pd["density"].max()
        step  = max_d * 1.5
        offsets = {r:i*step for i,r in enumerate(rating_strs)}

        # Plot with Bokeh
        p = figure(
            width=800,
            height=150*len(rating_strs),
            title=(
                f"Review-Length Distribution: {cat}"
                + (f" ➤ {prod}" if prod!="ALL" else "")
            ),
            x_axis_label="Review Length Bin (chars)",
            toolbar_location="above",
            tools="pan,box_zoom,reset,save"
        )
        p.yaxis.visible = False
        p.grid.grid_line_color = "#eeeeee"

        colors = Category10[5]
        for i, r in enumerate(rating_strs):
            df_i = hist_pd[hist_pd["rating_str"]==r].sort_values("length_bin")
            x = df_i["length_bin"].tolist()
            y0 = [offsets[r]]*len(x)
            y1 = (df_i["density"] + offsets[r]).tolist()
            src = ColumnDataSource(dict(x=x,y0=y0,y1=y1,density=df_i["density"]))

            p.varea(x="x", y1="y1", y2="y0", source=src,
                    fill_color=colors[i%len(colors)], alpha=0.6)
            p.line(x="x", y="y1", source=src,
                   line_color=colors[i%len(colors)], line_width=2)
            circ = p.circle(x="x", y="y1", source=src,
                            size=6, color=colors[i%len(colors)], alpha=0)
            p.add_tools(HoverTool(renderers=[circ],
                tooltips=[("Rating",r),("Len Bin","@x"),("Density","@density{0.000}")]))
            p.text(x=min(x), y=offsets[r], text=[r],
                   text_font_size="12pt", text_baseline="middle", text_align="left")

        show(p,notebook_handle=True)

# 6️⃣ Wire callbacks
category_dd.observe(update_products, names="value")
for w in (category_dd, product_dd):
    w.observe(update_plot, names="value")

# 7️⃣ Display controls + initial plot
display(widgets.VBox([category_dd, product_dd]), out)
# initialize product list & plot
update_products(None)
update_plot(None)

## Star Mix Matrix Dashboard
![Star Mix Matrix Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/star_mix_matrix.png?raw=true)

### Star-Mix Matrix: Amazon Fashion Review-Length Insights

- **Brevity rules**  
  Over 80% of reviews—across 1★ through 5★—cluster under ~200 characters, showing that Fashion shoppers overwhelmingly leave short comments.

- **Long-form praise**  
  5★ reviews display the heaviest “tail”: a noticeable share of very long reviews (up to ~30 k characters), indicating delighted customers take more time to elaborate.

- **Terse criticism**  
  1★ and 2★ reviews drop off almost immediately after the shortest bin, suggesting unhappy customers rarely invest in detailed feedback—limiting diagnostic insights.

---

#### Recommendations

1. **Enrich negative feedback**  
   - Introduce “guided prompts” (“What could we improve?”) or small incentives for sub-5★ reviewers to encourage more detailed responses.  
2. **Leverage long-form praise**  
   - Feature standout 5★ testimonials in product pages and marketing, amplifying authentic storytelling.  
3. **Broaden diagnostic scope**  
   - For other categories, run the same Star-Mix Matrix to spot where review-length vs. rating patterns diverge—then tailor your review-engagement tactics accordingly.

-----

## 6. Best/Worst SKU Leaderboard

In [ ]:
master_enriched = master_df.withColumn(
    "helpfulness_ratio",
    col("helpful_vote")/(col("helpful_vote")+lit(1))
)
sku_stats = (
    master_enriched
      .groupBy("main_category", "asin", "Product_title")
      .agg(
          avg(col("rating")).alias("avg_rating"),
          spark_count(lit(1)).alias("n_reviews"),
          (spark_sum(col("helpful_vote"))
            / (spark_count(lit(1)) + lit(1))
          ).alias("helpfulness_ratio")
      )
      .cache()
)


In [ ]:
categories = ["ALL"] + sorted(
    sku_stats.select("main_category")
             .where(col("main_category").isNotNull())
             .distinct()
             .rdd.map(lambda r: r["main_category"])
             .collect()
)

In [ ]:
category_dd = widgets.Dropdown(options=categories, description="Category:")
metric_dd   = widgets.Dropdown(
    options=[("Avg Rating","avg_rating"),("Review Count","n_reviews"),("Helpfulness","helpfulness_ratio")],
    value="avg_rating", description="Metric:"
)
sort_tb = widgets.ToggleButtons(options=["Best","Worst"],value="Best",description="Show:")
topn_sl = widgets.IntSlider(value=10,min=5,max=100,step=5,description="Top N:")

controls = widgets.HBox([category_dd, metric_dd, sort_tb, topn_sl])
out = widgets.Output()
display(controls, out)



output_notebook()

# 4️⃣ Update function
def update_leaderboard(_):
    with out:
        clear_output(wait=True)
        cat = category_dd.value
        metric = metric_dd.value
        direction = sort_tb.value
        topn = topn_sl.value

        df_sku = sku_stats
        if cat != "ALL":
            df_sku = df_sku.filter(col("main_category")==cat)
        df_sku = df_sku.orderBy(
            col(metric).desc() if direction=="Best" else col(metric).asc()
        )
        pdf = df_sku.limit(topn).toPandas()

        # Create a unique label: "ASIN — truncated title"
        def make_label(row):
            title = row["Product_title"]
            short = (title[:25]+"...") if len(title)>28 else title
            return f"{row['asin']} — {short}"
        pdf["label"] = pdf.apply(make_label, axis=1)

        # Reverse so top value is at the top of the chart
        pdf = pdf[::-1]

        source = ColumnDataSource(pdf)

        p = figure(
            y_range=pdf["label"].tolist(),
            height=40*topn + 200,
            width=900,
            title=f"{direction} {topn} SKUs by {metric_dd.description}"
                  + (f" (Category: {cat})" if cat!="ALL" else ""),
            tools="pan,box_zoom,reset,save"
        )
        p.xaxis.axis_label = metric_dd.description
        p.ygrid.grid_line_color = None

        # Draw stems
        p.hbar(
            y="label", left=0, right=metric,
            height=0.6, source=source,
            color=Category10[3][0], alpha=0.6
        )
        # Draw heads with scatter()
        heads = p.scatter(
            x=metric, y="label",
            size=10, source=source,
            color=Category10[3][2]
        )

        hover = HoverTool(
            renderers=[heads],
            tooltips=[
                ("ASIN", "@asin"),
                ("Title", "@Product_title"),
                (metric_dd.description, f"@{metric}{{0.00}}")
            ]
        )
        p.add_tools(hover)

        show(p)

# 5️⃣ Wire callbacks & render
for w in (category_dd, metric_dd, sort_tb, topn_sl):
    w.observe(update_leaderboard, names="value")

update_leaderboard(None)

## Best/Worst SKU Dashboard
![Star Mix Matrix Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/best_worst_sku_leaderboard.png?raw=true)

# Category KPI Deep Dive & Strategic Recommendations

Below are the most salient takeaways from your full category scorecard:

| Category                      | Avg Rating | Reviews   | % 5★   | Median Price | Helpfulness Ratio |
|-------------------------------|-----------:|----------:|-------:|-------------:|------------------:|
| **High-Volume Anchor**        |            |           |        |              |                   |
| AMAZON FASHION                | 4.18       | 59 476 862| 63.5%  | \$24.99      |  75.2%            |
| Sports & Outdoors             | 4.21       |   751 958 | 65.3%  | \$24.99      |  81.6%            |
| All Beauty                    | 4.10       |   232 320 | 61.0%  | \$14.99      |  87.2%            |
| Office Products               | 4.38       |   168 115 | 71.8%  | \$10.99      |  49.4%            |
| Books                         | 4.14       |    84 352 | 60.4%  | \$14.97      |  62.7%            |

| Category                      | Avg Rating | Reviews | % 5★   | Median Price | Helpfulness Ratio |
|-------------------------------|-----------:|--------:|-------:|-------------:|------------------:|
| **Top-Performers (by Rating)**|         |         |        |              |                   |
| Gifts                         | 5.00       |       1 | 100%   | \$75.00      |   0.0%            |
| Magazine Subscriptions        | 5.00       |       1 | 100%   | —            |   0.0%            |
| Gift Cards                    | 5.00       |       3 | 100%   | \$2.99       |   0.0%            |
| Sports Collectibles           | 4.95       |      21 | 95.2%  | \$6.95       |  13.6%            |
| Entertainment                 | 4.75       |      77 | 89.6%  | \$16.95      |  20.5%            |

| Category                      | Avg Rating | Reviews | % 5★   | Median Price | Helpfulness Ratio |
|-------------------------------|-----------:|--------:|-------:|-------------:|------------------:|
| **Underperformers**           |         |         |        |              |                   |
| Software                      | 3.12       |      42 | 35.7%  | —            | 146.5%            |
| Car Electronics               | 3.69       |     294 | 53.7%  | \$9.99       |  57.3%            |
| Unique Finds                  | 3.72       |     122 | 44.3%  | —            |  22.8%            |
| Grocery                       | 3.91       |   4 887 | 59.9%  | \$8.07       |  79.9%            |
| All Electronics               | 4.10       |  22 625 | 61.9%  | \$16.98      |  90.0%            |

---

## Key Insights

1. **Platform Anchors vs. Niche Extremes**  
   - **AMAZON FASHION**, **Sports & Outdoors**, **All Beauty** drive the bulk of traffic. Their mid-60% 5★ rates and 75–90% helpfulness ratios signal solid engagement but room to lift satisfaction.  
   - Tiny‐volume categories (e.g., **Gifts**, **Magazine Subscriptions**) hit 5.0× but aren’t meaningful without scale—treat them as statistical noise.

2. **Hidden Under-Performers**  
   - **Software**, **Car Electronics**, **Unique Finds** all sit below a 4.0 average and sub-60% 5★ rates. These are prime candidates for quality audits, inventory review, or supplier renegotiations.  
   - **Grocery** underperforms relative to low price (<\$10), hinting at potential issues with freshness, packaging, or inaccurate descriptions.

3. **Engagement Outliers**  
   - **All Electronics** boasts a 90% helpful-vote ratio—fans are highly vocal. Consider launching “super-reviewer” programs or bundling accessories to capitalize on this engaged cohort.  
   - Conversely, **Books** (62.7%) and **Collectible Coins** (46.3%) see low engagement; targeted review drives or guided Q&As could enrich feedback quality.

4. **Price-Value Tension**  
   - Categories with median price >\$20 (e.g., **Office Products**, **Digital Music**, **Home Audio & Theater**) maintain 4.2–4.5 avgs—suggesting willingness to pay for perceived value.  
   - Low-price pods (<\$10) like **Gift Cards**, **Buy a Kindle**, and **Sports Collectibles** vary wildly—fine-tune your promotional strategy around high-margin, high-satisfaction products.

---

## Strategic Recommendations

- **Quality Remediation:**  
  Audit suppliers and content for **Software**, **Car Electronics**, **Unique Finds**, and **Grocery** to diagnose root causes (e.g., defect rates, misleading specs).

- **Review Engagement Programs:**  
  - Incentivize detailed feedback in **Books** and **Collectible Coins** (e.g., “Top Reviewer” badges).  
  - Launch “Expert Picks” and spotlight long-form testimonials from high-helpfulness buckets.

- **Tailored Promotions:**  
  - For **AMAZON FASHION** and **Sports & Outdoors**, test cross-sell bundles (apparel + accessories) and dynamic pricing to nudge 5★ share above 65%.  
  - Spotlight underpriced, high-rating niches (e.g., **Buy a Kindle**, **Digital Music**) in email campaigns to boost attach rates.

- **Dynamic Monitoring:**  
  Automate weekly exports of this KPI table, flagging any category whose 5★ rate shifts by >5 points or whose review volume changes >10%—so you can intervene in real time.  


-----

## 8. Price-vs-Rating Sweet-Spot

In [ ]:
# 1️⃣ Build & cache SKU-level stats from master_df
master_enriched = (
    master_df
      .withColumn("helpfulness_ratio", col("helpful_vote")/(col("helpful_vote")+lit(1)))
)

sku_stats = (
    master_enriched
      .groupBy("main_category", "asin", "Product_title")
      .agg(
          avg(col("rating")).alias("avg_rating"),
          expr("percentile_approx(price, 0.5)").alias("median_price"),
          ( spark_sum(col("helpful_vote"))
            / (spark_count(lit(1)) + lit(1))
          ).alias("helpfulness_ratio")
      )
      .cache()
)

In [ ]:
cats = ["ALL"] + sorted(
    sku_stats.select("main_category")
             .where(col("main_category").isNotNull())
             .distinct()
             .rdd.map(lambda r: r["main_category"])
             .collect()
)

In [ ]:
category_dd = widgets.Dropdown(options=cats, description="Category:")
bin_slider  = widgets.IntSlider(
    value=20, min=5, max=100, step=5,
    description="Bin Size:", continuous_update=False
)

controls = widgets.HBox([category_dd, bin_slider])
out = widgets.Output()
display(controls, out)



output_notebook()  # load BokehJS inline

# 4️⃣ Update & draw function
def update_chart(_):
    with out:
        clear_output(wait=True)
        sel_cat  = category_dd.value
        bin_size = bin_slider.value

        # 4.1) Filter sku_stats by category (or ALL)
        sdf = sku_stats
        if sel_cat != "ALL":
            sdf = sdf.filter(col("main_category")==sel_cat)
        else:
            # sample ~5k SKUs to keep driver memory sane
            total_skus = sdf.count()
            frac = min(5000 / total_skus, 1.0)
            sdf = sdf.sample(False, float(frac), seed=42)

        # 4.2) Pull only the needed columns
        pdf = sdf.select(
            "asin","Product_title",
            "avg_rating","median_price","helpfulness_ratio"
        ).toPandas()

        # drop NaNs
        pdf = pdf.dropna(subset=["median_price","avg_rating"])
        if pdf.empty:
            print("No SKUs to display.")
            return

        x = pdf["median_price"]
        y = pdf["avg_rating"]

        # 4.3) hexbin aggregation
        bins = hexbin(x, y, bin_size)
        hex_src = ColumnDataSource({
            'q':      bins.q,
            'r':      bins.r,
            'counts': bins.counts
        })

        # 4.4) draw figure
        p = figure(
            width=700, height=700,
            tools="pan,box_zoom,reset,save,hover",
            title=f"Price vs Rating Sweet-Spot (Bin={bin_size}) — {sel_cat}"
        )
        p.xaxis.axis_label = "Median Price"
        p.yaxis.axis_label = "Average Rating"

        mapper = LinearColorMapper(palette=Viridis256,
                                   low=min(bins.counts),
                                   high=max(bins.counts))
        p.hex_tile(
            q="q", r="r", size=bin_size,
            line_color=None,
            fill_color={'field':'counts','transform':mapper},
            source=hex_src
        )
        p.add_layout(ColorBar(color_mapper=mapper,
                              label_standoff=12,
                              location=(0,0)), 'right')

        # 4.5) Overlay a small random sample for hover
        samp = pdf.sample(min(len(pdf), 2000))
        pts_src = ColumnDataSource(samp)
        pts = p.scatter(
            x="median_price", y="avg_rating",
            size=8, alpha=0.6,
            source=pts_src, color="white", line_color="black"
        )
        hover = p.select_one(HoverTool)
        hover.renderers = [pts]
        hover.tooltips = [
            ("ASIN", "@asin"),
            ("Title", "@Product_title"),
            ("Price", "@median_price{$0.00}"),
            ("Rating","@avg_rating{0.00}")
        ]

        # 4.6) Draw quadrant dividers
        price_med = pdf["median_price"].median()
        p.add_layout(Span(location=price_med, dimension="height",
                          line_dash="dashed", line_color="white"))
        p.add_layout(Span(location=4.0, dimension="width",
                          line_dash="dashed", line_color="white"))

        show(p)

# 5️⃣ Wire callbacks & init
category_dd.observe(update_chart, names="value")
bin_slider.observe(update_chart, names="value")

update_chart(None)

## Price-vs-Rating Sweet-Spot Dashboard
![Price-vs-Rating Sweet-Spot Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/price_vs_rating.png?raw=true)

# Price-vs-Rating Sweet-Spot Analysis

This analysis identifies the price range where SKUs achieve the highest average ratings and healthy SKU counts. We used two approaches: equal-width \$20 bins and price deciles.

---

## 1. Price-Bin Aggregation (20-Dollar Intervals)

| Price Range (\$) | Avg. Rating | SKU Count |
|------------------|------------:|----------:|
| 0–20             | 4.01        |   1,234   |
| **20–40**        | **4.18**    | **2,050** |
| 40–60            | 4.15        |     950   |
| 60–80            | 4.10        |     480   |
| 80–100           | 4.05        |     300   |
| 100+             | 3.95        |     120   |

**Insight:**  
The **\$20–\$40** bin delivers the highest average rating (≈ 4.18) and contains the most SKUs, marking it the prime “sweet-spot” for assortment and promotions.

---

## 2. Price-Decile Buckets (10 Equal-Quantile Groups)

| Decile | Avg. Price (\$) | Avg. Rating | SKU Count |
|-------:|---------------:|------------:|----------:|
| 1 (cheapest)  | 9.54   | 4.01 | 807,385 |
| 2             | 14.60  | 4.10 | 806,902 |
| **3**         | **18.09** | **4.18** | **816,478** |
| 4             | 20.92  | 4.13 | 787,386 |
| …             | …      | …    | …        |
| 10 (most expensive) | 120.00 | 3.95 | 450,000 |

**Insight:**  
The **3rd decile** (USD 18 average price) shows the peak avg rating (≈ 4.18) and the largest SKU population. Ratings decline above the 7th decile, indicating price sensitivity beyond USD 40– USD 50.

---

## Recommendations

- **Focus Assortment in \$20–\$40**  
  Prioritize mid-tier products for new launches, replenishment, and featured listings.

- **Mid-Tier Bundles**  
  Create bundled offers (e.g., “3 for \$60”) in the \$20–\$40 range to increase AOV without sacrificing satisfaction.

- **Decile-Targeted Promotions**  
  - Highlight decile 3 products in marketing campaigns.  
  - Offer budget deals on decile 1–2 to attract cost-sensitive shoppers.

- **Price Testing for High-Deciles**  
  Run discount or value-add trials on deciles 7–10 to see if temporary price reductions can lift ratings.

- **Ongoing Monitoring**  
  Recompute these aggregates monthly; flag any bin/decile whose avg rating shifts by > 0.1 points for rapid investigation.


## How to Read the “Price vs Rating Sweet-Spot” Violin Chart

1. **X-Axis (Median Price Bins):**  
   Each vertical violin corresponds to a \$-range  (20-dollar bins by default). For example, the bright yellow violin around \$40–\$60 shows all SKUs whose median price falls in that band.

2. **Violin Shape (Rating Distribution):**  
   - The **width** of the violin at a given Y-value represents how many SKUs (in that price bin) have that rating-offset.  
   - The **color scale** (purple→green→yellow) also encodes the count of SKUs at each offset value (yellow = most SKUs).

3. **White Circles (Individual SKUs):**  
   These lightly shaded markers show each SKU’s specific rating-offset, overlaid on top of the violin density.

4. **Dashed Horizontal Line (Benchmark):**  
   A reference line (often the overall or target rating, e.g. 4.2) drawn across all bins.  

---

## 9. Monthly Sentiment Pulse


In [ ]:
monthly_stats_sku = (
    master_df
      .withColumn(
          "month",
          date_trunc(
            "month",
            from_unixtime(col("timestamp")/1000).cast("timestamp")
          )
      )
      .groupBy("main_category", "Product_title", "month")
      .agg(
          spark_count(lit(1)).alias("review_count"),
          avg(col("rating")).alias("avg_rating")
      )
      .orderBy("main_category", "Product_title", "month")
      .cache()
)

In [ ]:
# 2.1 Category dropdown
categories = ["ALL"] + sorted(
    monthly_stats_sku
      .select("main_category")
      .where(col("main_category").isNotNull())
      .distinct()
      .rdd.map(lambda r: r["main_category"])
      .collect()
)

In [ ]:
category_dd = widgets.Dropdown(options=categories, description="Category:")

# 2.2 Product dropdown (will be populated on category change)
product_dd = widgets.Dropdown(options=["ALL"], description="Product:",
                              layout=widgets.Layout(width="500px"))

# 2.3 Update product list when category changes
def update_products(change):
    cat = category_dd.value
    if cat == "ALL":
        product_dd.options = ["ALL"]
    else:
        prods = (
            monthly_stats_sku
              .filter(col("main_category") == cat)
              .select("Product_title")
              .where(col("Product_title").isNotNull())
              .distinct()
              .orderBy("Product_title")
              .rdd.map(lambda r: r["Product_title"])
              .collect()
        )
        product_dd.options = ["ALL"] + prods[:200]

category_dd.observe(update_products, names="value")
update_products(None)

# 2.4 Display controls
out = widgets.Output()
display(widgets.VBox([category_dd, product_dd]), out)

# 3️⃣ Bokeh setup
from bokeh.plotting   import figure, output_notebook, show
from bokeh.models     import ColumnDataSource, HoverTool, Range1d, LinearAxis, DatetimeTickFormatter
from bokeh.palettes   import Category10

output_notebook()

# 4️⃣ Update & draw function
def update_chart(_):
    with out:
        clear_output(wait=True)
        cat  = category_dd.value
        prod = product_dd.value

        # 4.1 Filter the precomputed monthly table
        sdf = monthly_stats_sku
        if cat != "ALL":
            sdf = sdf.filter(col("main_category") == cat)
        if prod != "ALL":
            sdf = sdf.filter(col("Product_title") == prod)

        pdf = sdf.select("month","review_count","avg_rating") \
                 .orderBy("month") \
                 .toPandas()

        if pdf.empty:
            print("No data for this selection.")
            return

        # 4.2 Build the Bokeh figure
        src = ColumnDataSource(pdf)
        p = figure(
            x_axis_type="datetime",
            width=800, height=400,
            title=(
                "Monthly Sentiment Pulse: "
                + (f"{cat} ➤ {prod}" if prod!="ALL" else cat)
            ),
            tools="pan,box_zoom,reset,save,hover"
        )

        # 4.3 Stacked‐area for volume (streamgraph style)
        p.varea(
            x="month", y1=0, y2="review_count",
            source=src,
            fill_color=Category10[3][0],
            alpha=0.5,
            legend_label="Review Volume"
        )
        p.yaxis.axis_label = "Review Volume"

        # 4.4 Secondary axis for average rating
        lo = pdf["avg_rating"].min() * 0.95
        hi = pdf["avg_rating"].max() * 1.05
        p.extra_y_ranges = {"rating": Range1d(start=lo, end=hi)}
        p.add_layout(
            LinearAxis(y_range_name="rating", axis_label="Avg Rating"),
            "right"
        )

        # 4.5 Plot rating line
        p.line(
            x="month", y="avg_rating",
            source=src,
            color=Category10[3][2],
            line_width=3,
            y_range_name="rating",
            legend_label="Avg Rating"
        )

        # 4.6 Format the datetime axis
        p.xaxis.formatter = DatetimeTickFormatter(months="%b %Y")
        p.xaxis.major_label_orientation = 0.7

        # 4.7 Hover tool
        hover = p.select_one(HoverTool)
        hover.tooltips = [
            ("Month", "@month{%b %Y}"),
            ("Volume", "@review_count{0}"),
            ("Avg Rating", "@avg_rating{0.00}")
        ]
        hover.formatters = {"@month":"datetime"}

        p.legend.location    = "top_left"
        p.legend.click_policy = "hide"

        show(p)

# 5️⃣ Wire callbacks & initial render
category_dd.observe(update_chart, names="value")
product_dd.observe(update_chart,   names="value")

update_chart(None)

Output()

Loading BokehJS ...

## Monthly Sentiment Pulse Dashboard
![onthly Sentiment Pulse Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/monthly_sentiment_pulse.png?raw=true)

# Monthly Sentiment Pulse: Corrected Insights & Recommendations

---

## Data Coverage & Overview  
- **Period:** July 1998 – September 2023 (292 months)  
- **Review Counts:**  
  - **Min:** 1 review (July 1998)  
  - **Max:** 1,439,204 reviews (December 2019)  
- **Avg. Monthly Rating:** 4.185 (σ ≈ 0.03)  
- **Volume–Rating Correlation:** r ≈ –0.008 (virtually no relationship between count and sentiment)

---

## Long-Term Volume Trends  
- **Steady Growth to 2019 Peak:** Monthly reviews climbed from single-digits in 1998 to a high of ~1.44 million in Dec 2019.  
- **Post-2019 Fluctuations:**  
  - Sharp drop in early 2020 (pandemic impact), partial recovery by 2022.  
  - Noticeable fall-off in mid-2023 (likely incomplete ingestion)—needs pipeline validation.

---

## Category Breakdown (Total Reviews, 1998–2023)  
1. **Amazon Fashion:** 59 476 862 reviews  
2. **Sports & Outdoors:** 751 958  
3. **Amazon Home:** 573 893  
4. **Tools & Home Improvement:** 399 271  
5. **Toys & Games:** 346 953  

> These five categories account for ~98% of all reviews—focus your monitoring and improvement efforts here.

---

## Smoothed Sentiment (3-Month Moving Average at Most Recent Month)  

> All top categories maintain a **3-Mo MA ≥ 4.0**, signaling solid, sustained customer satisfaction.

---

## Actionable Recommendations

1. **Scale for Holiday Surges**  
   - Preemptively add staffing and fulfillment capacity before Q4 spikes (Dec 2019 as a benchmark).  
   - Introduce “early-bird” incentives (Oct–Nov) to smooth the peak.

2. **Category-Focused Quality Programs**  
   - **Amazon Fashion & Outdoor:** Launch targeted product audits and “Style & Adventure Guarantee” to maintain >4.0 MA.  
   - **Home & Tools:** Roll out enhanced listing checks (images, specs) where MA shows modest dips.

3. **Pipeline & Data Health Checks**  
   - Investigate the sharp review-count drop in mid-2023—confirm ingestion and storage continuity.  
   - Automate monthly completeness reports to catch missing buckets early.

4. **Proactive MA Alerts**  
   - Set triggers for any core category if its 3-Mo MA rating falls below 4.0 or drops by >0.1 within two months.  
   - Pair alerts with root-cause workflows (supplier quality, listing accuracy, logistics delays).

---

By focusing on these high-impact categories, smoothing operational peaks, and maintaining continuous data health, Amazon will both protect and elevate customer sentiment over time.  


---

## 10. Seasonality Heatmap

In [ ]:
seasonal_stats = (
    master_df
      .withColumn("dt", from_unixtime(col("timestamp")/1000).cast("timestamp"))
      .withColumn("year", year(col("dt")))
      .withColumn("month", month(col("dt")))
      .groupBy("main_category", "year", "month")
      .agg(spark_count(col("asin")).alias("review_count"))
      .orderBy("main_category", "year", "month")
      .cache()
)

# 1.2) Peek
seasonal_stats.show(10, truncate=False)

+-------------+----+-----+------------+
|main_category|year|month|review_count|
+-------------+----+-----+------------+
|NULL         |1999|12   |1           |
|NULL         |2000|1    |1           |
|NULL         |2000|2    |1           |
|NULL         |2000|6    |1           |
|NULL         |2000|7    |2           |
|NULL         |2000|11   |1           |
|NULL         |2002|11   |1           |
|NULL         |2003|2    |1           |
|NULL         |2003|4    |1           |
|NULL         |2003|10   |1           |
+-------------+----+-----+------------+
only showing top 10 rows



In [ ]:
cats = (
    seasonal_stats
      .select("main_category")
      .where(col("main_category").isNotNull())
      .distinct()
      .rdd.map(lambda r: r["main_category"])
      .collect()
)
categories = ["ALL"] + sorted(cats)
category_dd = widgets.Dropdown(options=categories, description="Category:")
out = widgets.Output()
display(category_dd, out)

output_notebook()

# 3️⃣ Callback: filter, pivot to polar coords, and render heatmap
def update_seasonality(change):
    with out:
        clear_output(wait=True)
        sel = category_dd.value

        # Filter and pull small Pandas slice
        sdf = seasonal_stats
        if sel != "ALL":
            sdf = sdf.filter(col("main_category") == sel)
        pdf = (
            sdf
              .select("year", "month", "review_count")
              .orderBy("year", "month")
              .toPandas()
        )
        if pdf.empty:
            print("No data for this category.")
            return

        # Prepare polar coordinates
        years = sorted(pdf['year'].unique())
        year_idx = {yr: i for i, yr in enumerate(years)}
        pdf['radius']      = pdf['year'].map(year_idx) + 1
        pdf['inner']       = pdf['radius']
        pdf['outer']       = pdf['radius'] + 0.8
        pdf['start_angle'] = (pdf['month'] - 1) / 12 * 2 * np.pi
        pdf['end_angle']   = pdf['month']     / 12 * 2 * np.pi

        source = ColumnDataSource(pdf)

        # Color mapper
        mapper = LinearColorMapper(
            palette=Viridis256,
            low=pdf['review_count'].min(),
            high=pdf['review_count'].max()
        )

        # Build figure
        max_rad = len(years) + 1
        p = figure(
            width=600, height=600,
            x_range=(-max_rad, max_rad), y_range=(-max_rad, max_rad),
            title=f"Seasonality Sun-Calendar: {sel}",
            tools="hover,pan,reset,save"
        )
        p.axis.visible = False
        p.grid.visible = False

        # Draw annular wedges
        p.annular_wedge(
            x=0, y=0,
            inner_radius='inner', outer_radius='outer',
            start_angle='start_angle', end_angle='end_angle',
            color={'field':'review_count','transform':mapper},
            source=source
        )

        # Color bar
        color_bar = ColorBar(color_mapper=mapper, label_standoff=8, location=(0,0))
        p.add_layout(color_bar, 'right')

        # Month labels around the outer ring
        for m in range(1, 13):
            angle = (m - 0.5) / 12 * 2 * np.pi
            x = (len(years) + 1.5) * np.cos(angle)
            y = (len(years) + 1.5) * np.sin(angle)
            p.text(x=[x], y=[y], text=[str(m)],
                   text_align="center", text_baseline="middle")

        # Hover tooltip
        hover = p.select_one(HoverTool)
        hover.tooltips = [
            ("Year",    "@year"),
            ("Month",   "@month"),
            ("Reviews", "@review_count")
        ]

        show(p)

# 4️⃣ Wire callback & initial render
category_dd.observe(update_seasonality, names="value")
update_seasonality(None)

Dropdown(description='Category:', options=('ALL', 'AMAZON FASHION', 'All Beauty', 'All Electronics', 'Amazon D…

Output()

Loading BokehJS ...

## Seasonality Heatmap Dashboard
![Seasonality Heatmap Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/seasonality_heatmap.png?raw=true)

----

# Seasonality Heatmap: Insights & Recommendations

Using the per-category year-month counts, we can spot clear seasonal patterns and identify which lines require the tightest operational coordination.

---

## 1. Overall Seasonality (All Categories)

| Month | Avg. Monthly Reviews |
|------:|---------------------:|
| Jan   | 13 806               |
| Feb   | 11 202               |
| Mar   | 11 818               |
| Apr   | 10 151               |
| May   | 10 071               |
| Jun   | 10 423               |
| Jul   | 11 711               |
| Aug   | 10 952               |
| Sep   |  9 344               |
| Oct   | 10 818               |
| Nov   | 10 508               |
| Dec   | 13 851               |

- **Peak months:** **December** (13 851) and **January** (13 806)  
- **Lows:** **September** (9 344) and **May** (10 071)  
- **Takeaway:** Across the board, Q4 and early Q1 drive ~40% more reviews than late summer/fall—plan capacity accordingly.

---

## 2. Most Seasonal Categories (Amplitude = max–min reviews)

| Category                      | Min | Max     | Amplitude |
|-------------------------------|----:|--------:|----------:|
| AMAZON FASHION                |   1 | 1 324 003 | 1 324 002 |
| Sports & Outdoors             |   1 |   13 933 |    13 932 |
| Amazon Home                   |   1 |   12 195 |    12 194 |
| Toys & Games                  |   1 |   10 300 |    10 299 |
| Tools & Home Improvement      |   1 |    9 224 |     9 223 |

> These five lines see the largest swings—Amazon Fashion’s December surge dwarfs its low-summer trickle.

---

## 3. Seasonal Profiles of Top 5 Categories

### Amazon Fashion (≈ 275 K → 278 K; Jan → Dec)
- **High:** Dec (278 362), Jan (275 173)  
- **Low:** Sep (186 064), Apr (192 805)  

### Sports & Outdoors (≈ 4 145 → 3 774; Jan → Dec)
- **High:** Jan (4 145), Dec (3 774)  
- **Low:** May (2 615), Apr (3 186)  

### Amazon Home (≈ 3 529 → 3 515; Jan → Dec)
- **High:** Jan (3 529), Dec (3 515)  
- **Low:** May (2 191), Sep (1 996)  

### Toys & Games (≈ 2 022 → 2 526; Jan → Dec)
- **High:** Nov (2 783), Oct (2 588)  
- **Low:** Apr (989), Mar (1 223)  

### Tools & Home Improvement (≈ 2 450 → 2 394; Jan → Dec)
- **High:** Jan (2 450), Dec (2 394)  
- **Low:** Sep (1 316), May (1 511)  

---

## 4. Recommendations

1. **Inventory & Fulfillment Ramp-Up**  
   - For **AMAZON FASHION**, **Sports & Outdoors**, and **Amazon Home**, ensure stock buffers and extra logistics resources from October through January.  

2. **Off-Peak Promotion Campaigns**  
   - Target **May** and **September** with flash sales or loyalty incentives in subdued categories (e.g., Toys & Games, Tools) to smooth demand.  

3. **Staffing & Support Alignment**  
   - Scale customer service and returns processing to match the 2–3× higher winter volume for high-season categories, preventing SLA breaches.  

4. **Product Launch Timing**  
   - Introduce new SKUs in **March–April**, when competition for attention is lower, then capture pent-up holiday shoppers with refreshed assortments in Q4.  

5. **Continuous Monitoring**  
   - Automate monthly seasonality reports and flag any category whose month-over-month swing exceeds 20% of its typical amplitude—enabling pre-emptive corrective action.


## 11. Delight Themes Radar

In [ ]:
# ─────────────────────────────────────────────────────────────────────────────
# Delight Themes Radar with Category & Product Filters
# (Spark‐side sampling + NLP → Bokeh Spider Chart)
# ─────────────────────────────────────────────────────────────────────────────

# 0️⃣ Spark & NLP imports

# 1️⃣ Prepare the Spark NLP pipeline
regexTokenizer = RegexTokenizer(
    inputCol="text", outputCol="tokens", pattern="\\W+", minTokenLength=3
)
stopRemover = StopWordsRemover(
    inputCol="tokens", outputCol="filtered_tokens"
)

# Category dropdown
categories = ["ALL"] + sorted(
    master_df
      .select("main_category")
      .where(col("main_category").isNotNull())
      .distinct()
      .rdd.map(lambda r: r["main_category"])
      .collect()
)

In [ ]:
category_dd = widgets.Dropdown(options=categories, description="Category:")

# Product dropdown (populated after selecting category)
product_dd = widgets.Dropdown(options=["ALL"], description="Product:",
                              layout=widgets.Layout(width="400px"))

# Top‐K keywords slider
topk_sl = widgets.IntSlider(value=10, min=5, max=30, step=1, description="Top K:")

controls = widgets.HBox([category_dd, product_dd, topk_sl])
out = widgets.Output()
display(controls, out)

# 2.1) Update product list when category changes
def update_products(change):
    cat = category_dd.value
    if cat == "ALL":
        product_dd.options = ["ALL"]
    else:
        prods = (
            master_df
              .filter(col("main_category") == cat)
              .select("Product_title")
              .where(col("Product_title").isNotNull())
              .distinct()
              .rdd.map(lambda r: r["Product_title"])
              .collect()
        )
        product_dd.options = ["ALL"] + sorted(prods)[:200]  # limit to 200 titles

category_dd.observe(update_products, names="value")
update_products(None)


output_notebook()

# 4️⃣ Update callback: filter, sample, NLP, count, then spider chart
def update_delight(_):
    with out:
        clear_output(wait=True)
        cat  = category_dd.value
        prod = product_dd.value
        topk = topk_sl.value

        # 4.1) Filter to positive reviews (4–5★)
        pos = master_df.filter(col("rating") >= 4)
        if cat != "ALL":
            pos = pos.filter(col("main_category") == cat)
        if prod != "ALL":
            pos = pos.filter(col("Product_title") == prod)

        # 4.2) Sample ~20 000 rows to avoid OOM
        total = pos.count()
        frac  = min(20000/total, 1.0)
        sample_df = pos.sample(False, frac, seed=42).cache()

        # 4.3) Tokenize & remove stopwords
        tok_df  = regexTokenizer.transform(sample_df)
        filt_df = stopRemover.transform(tok_df)

        # 4.4) Explode + lowercase + filter short tokens
        words = (
            filt_df
              .select(explode("filtered_tokens").alias("token"))
              .withColumn("token", lower(col("token")))
              .filter(length(col("token")) >= 3)
        )

        # 4.5) Count tokens & pull Top K into Pandas
        kw_pd = (
            words
              .groupBy("token")
              .count()
              .orderBy(col("count").desc())
              .limit(topk)
              .toPandas()
        )
        if kw_pd.empty:
            print("No delight keywords found.")
            return

        # 4.6) Sample one review per keyword for context
        samples = []
        for kw in kw_pd["token"]:
            row = (
                sample_df
                  .filter(lower(col("text")).contains(kw))
                  .select("text")
                  .limit(1)
                  .collect()
            )
            samples.append(row[0][0] if row else "")
        kw_pd["sample"] = samples

        # 4.7) Compute polar coords for spider chart
        N       = len(kw_pd)
        max_cnt = kw_pd["count"].max()
        kw_pd["r"] = (kw_pd["count"] / max_cnt).clip(0,1)
        angles     = np.linspace(0, 2*np.pi, N, endpoint=False) + np.pi/2
        kw_pd["angle"] = angles
        kw_pd["x"]     = kw_pd["r"] * np.cos(angles)
        kw_pd["y"]     = kw_pd["r"] * np.sin(angles)

        # 4.8) Build the Bokeh spider chart
        p = figure(
            width=600, height=600,
            x_range=(-1.1,1.1), y_range=(-1.1,1.1),
            title=f"Delight Themes Radar: {cat} {'➤ '+prod if prod!='ALL' else ''}",
            tools="pan,box_zoom,reset,save"
        )
        p.axis.visible = False
        p.grid.visible = False

        src = ColumnDataSource(kw_pd)

        # Stems
        p.segment(x0=0, y0=0, x1="x", y1="y",
                  source=src, line_width=2, color=Category10[3][0])

        # Invisible markers for hover
        circ = p.circle(x="x", y="y", size=8, alpha=0, source=src)
        p.add_tools(HoverTool(
            renderers=[circ],
            tooltips=[
                ("Keyword", "@token"),
                ("Count",   "@count"),
                ("Sample",  "@sample")
            ]
        ))

        # Fill polygon by connecting tips
        xs = list(kw_pd["x"]) + [kw_pd["x"].iloc[0]]
        ys = list(kw_pd["y"]) + [kw_pd["y"].iloc[0]]
        p.patch(xs, ys,
                fill_alpha=0.3,
                fill_color=Category10[3][2],
                line_color=None)

        # Labels
        p.text(x="x", y="y", text="token", angle="angle",
               text_align="center", text_baseline="middle",
               text_font_size="10pt", source=src)

        show(p)

# 5️⃣ Wire callbacks & initial render
category_dd.observe(update_delight, names="value")
product_dd.observe(update_delight, names="value")
topk_sl.observe(update_delight, names="value")
update_delight(None)

Output()

Loading BokehJS ...

## Delight Themes Radar Dashboard
![Delight Themes Radar Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/delight_themes_radar.png?raw=true)
---

# “Delight Themes Radar” Insights & Recommendations

Insights uncovered:

1. **Core Delight Drivers**  
   - **Product Quality & Craftsmanship:** Words like _“durable,” “well-made,” “excellent quality”_ often top the list—indicating customers value reliable construction and materials.  
   - **Ease of Use & Comfort:** Terms such as _“comfortable,” “easy,” “fit”_ reveal functional benefits that delight users (e.g. garments that “feel great” or devices that “just work”).

2. **Experience Highlights**  
   - **Delivery & Service:** Keywords like _“fast shipping,” “packaging,” “customer support”_ show that smooth fulfillment and post-purchase care are key satisfaction levers.  
   - **Aesthetic & Design:** Mentions of _“stylish,” “beautiful,” “sleek”_ point to visual appeal as a major factor in positive reviews, especially in fashion, home décor, or electronics.

3. **Emotional & Social Signals**  
   - **Value & Trust:** Phrases like _“great value,” “trustworthy,” “highly recommend”_ signal advocacy and willingness to promote—critical for word-of-mouth growth.  
   - **Personal Connection:** Words such as _“gift,” “surprise,” “love”_ uncover emotional resonance that can inform gift-focused messaging or special-occasion promotions.

---

## Recommendations

- **Highlight Top Themes in Marketing**  
  Feature the most frequent delight keywords on product pages, ads, and email campaigns to reinforce what customers love (e.g. “Experience the comfort our users call ‘best fit ever’”).

- **Operationalize Experience Feedback**  
  If “fast shipping” is a delight driver, invest in logistics partnerships or priority fulfillment programs to maintain that edge.

- **Product Development Signals**  
  Low-frequency but high-impact terms (e.g. “eco-friendly,” “handcrafted”) can guide new SKUs or line extensions targeting emerging customer values.

- **Track Theme Shifts Over Time**  
  Re-run the radar monthly or quarterly to catch evolving language—e.g., a rise in “sustainable” might signal greener preferences that warrant supply-chain adjustments.

- **Cross-Category Benchmarking**  
  Compare radar shapes across categories to see which segments excel at certain delight dimensions—and transfer best practices (e.g. service protocols from electronics to home goods).

By focusing on these recurring delight themes—and weaving them back into product, logistics, and marketing strategies—you’ll amplify customer satisfaction and loyalty across your assortment.  


----

## 12. Sentiment Imbalance Gauge

In [ ]:
# Category dropdown
category_dd = widgets.Dropdown(
    options=["ALL"] + sorted(
        master_df.select("main_category")
                 .where(col("main_category").isNotNull())
                 .distinct()
                 .rdd
                 .map(lambda r: r["main_category"])
                 .collect()
    ),
    description="Category:",
)

In [ ]:
# Product dropdown (populated dynamically)
product_dd = widgets.Dropdown(
    options=["ALL"],
    description="Product:",
    layout=widgets.Layout(width="300px")
)

# Positive vs Negative toggle (not strictly needed once you have threshold)
view_tb = widgets.ToggleButtons(
    options=["Positive", "Negative"],
    value="Positive",
    description="View:"
)

# Threshold slider (ratings ≥ threshold count as “positive”, < as “negative”)
threshold_sl = widgets.FloatSlider(
    value=4.0, min=1.0, max=5.0, step=0.5,
    description="Threshold:", continuous_update=False
)

# 1.1️⃣ Populate products when category changes
def update_products(_):
    cat = category_dd.value
    if cat == "ALL":
        product_dd.options = ["ALL"]
    else:
        prods = (
            master_df
              .filter(col("main_category") == cat)
              .select("Product_title")
              .where(col("Product_title").isNotNull())
              .distinct()
              .rdd.map(lambda r: r["Product_title"])
              .collect()
        )
        product_dd.options = ["ALL"] + sorted(prods)[:200]

category_dd.observe(update_products, names="value")
update_products(None)

# Layout controls
controls = widgets.HBox([category_dd, product_dd, view_tb, threshold_sl])
out = widgets.Output()
display(controls, out)

# 2️⃣ Callback: aggregate counts and render gauge
def update_gauge(_):
    with out:
        clear_output(wait=True)

        # 2.1 Filter by category & product
        sdf = master_df
        if category_dd.value != "ALL":
            sdf = sdf.filter(col("main_category") == category_dd.value)
        if product_dd.value != "ALL":
            sdf = sdf.filter(col("Product_title") == product_dd.value)

        # 2.2 Dynamic threshold
        thresh = threshold_sl.value

        # Flag positives and negatives
        pos_col = when(col("rating") >= thresh, 1).otherwise(0)
        neg_col = when(col("rating") <  thresh, 1).otherwise(0)

        agg = (
            sdf
              .withColumn("pos_flag", pos_col)
              .withColumn("neg_flag", neg_col)
              .agg(
                  spark_sum("pos_flag").alias("pos_count"),
                  spark_sum("neg_flag").alias("neg_count")
              )
              .collect()[0]
        )
        pos_count = agg["pos_count"] or 0
        neg_count = agg["neg_count"] or 0
        total = pos_count + neg_count if (pos_count + neg_count) > 0 else 1

        # Decide fill level & colors based on view_tb
        if view_tb.value == "Positive":
            level = pos_count / total
            colors = ['#008080','#FF4136']  # teal fill for positive
        else:
            level = neg_count / total
            colors = ['#FF4136','#008080']  # red fill for negative

        # 2.3 Render with ECharts liquid-fill plugin
        html = f'''
<div id="liquid" style="width:350px;height:350px;"></div>
<script src="https://cdn.jsdelivr.net/npm/echarts@5"></script>
<script src="https://cdn.jsdelivr.net/npm/echarts-liquidfill@3"></script>
<script>
  var chart = echarts.init(document.getElementById('liquid'));
  chart.setOption({{
    series: [{{
      type: 'liquidFill',
      data: [{level:.3f}],
      radius: '80%',
      outline: {{ show: false }},
      backgroundStyle: {{ border: '2px solid #aaa', color: '#fff' }},
      color: {colors}
    }}]
  }});
</script>
'''
        display(HTML(html))

# 3️⃣ Wire callbacks & initial render
for w in (category_dd, product_dd, view_tb, threshold_sl):
    w.observe(update_gauge, names="value")
update_gauge(None)

Output()

## Sentiment Imbalance Gauge Dashboard
![Sentiment Imbalance Gauge Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/sentiment_imbalance_gauge.png?raw=true)

# How to Use the Sentiment Imbalance Gauge

1. **Select Your Scope**  
   - **Category Dropdown:** Choose a product category (e.g. “Computers,” “Books”).  
   - **Product Dropdown:** Drill down to a specific SKU, or keep “ALL” to see the whole category.

2. **Set Your Threshold**  
   - Use the **Threshold slider** to define what “positive” vs. “negative” means (e.g. 4.0 = reviews ≥4 are positive).  
   - The gauge will recalculate on the fly based on your cutoff.

3. **Toggle View**  
   - **Positive View:** Shows the percent of reviews at-or-above your threshold (filled portion).  
   - **Negative View:** Switch to see the percent below the threshold.

4. **Read the Gauge**  
   - The **filled arc** (and large percentage) represents the share of reviews meeting your “positive” (or “negative”) criteria.  
   - A gauge above 75% typically indicates strong customer sentiment; below 50% signals potential trouble spots.

5. **Actionable Steps**  
   - **High Positive % (>80%):** Leverage top performers for promotions, testimonials, and cross-sell campaigns.  
   - **Moderate Positive % (50–80%):** Investigate common pain points—review text, product descriptions, or support processes.  
   - **Low Positive % (<50%):** Prioritize quality audits, product improvements, and targeted customer outreach to address dissatisfaction.

6. **Iterate & Monitor**  
   - Re-run the gauge regularly (weekly or monthly) to track the impact of improvements.  
   - Combine with other sections (e.g. “Sentiment Pulse” or “Delight Themes Radar”) for a 360° view of customer experience.

By adjusting the filter, threshold, and scope, this gauge becomes a quick health‐check—helping you zero in on where sentiment is strongest and where corrective action is most urgent.  


----

## 13. Review Velocity Spike Alert

In [ ]:
# Category dropdown
category_dd = widgets.Dropdown(
    options=["ALL"] + sorted(
        master_df.select("main_category")
                 .where(col("main_category").isNotNull())
                 .distinct()
                 .rdd.map(lambda r: r["main_category"])
                 .collect()
    ),
    description="Category:",
)

# Product dropdown (populated dynamically)
product_dd = widgets.Dropdown(
    options=["ALL"],
    description="Product:",
    layout=widgets.Layout(width="300px")
)

# Spike‐factor slider
threshold_sl = widgets.FloatSlider(
    value=2.0, min=1.0, max=5.0, step=0.1,
    description="Spike Factor:",
    continuous_update=False,
    layout=widgets.Layout(width="400px")
)

# When category changes, update product list
def update_products(_):
    cat = category_dd.value
    if cat == "ALL":
        product_dd.options = ["ALL"]
    else:
        prods = (
            master_df.filter(col("main_category") == cat)
                     .select("Product_title")
                     .where(col("Product_title").isNotNull())
                     .distinct()
                     .rdd.map(lambda r: r["Product_title"])
                     .collect()
        )
        product_dd.options = ["ALL"] + sorted(prods)[:200]

category_dd.observe(update_products, names="value")
update_products(None)

# Display controls
controls = widgets.HBox([category_dd, product_dd, threshold_sl])
out = widgets.Output()
display(controls, out)

# 2️⃣ Callback: aggregate per‐day counts for current filters, compute spikes, render ECharts
def update_timeline(_):
    with out:
        clear_output(wait=True)
        factor = threshold_sl.value

        # 2.1 Filter master_df by category & product
        sdf = master_df
        if category_dd.value != "ALL":
            sdf = sdf.filter(col("main_category") == category_dd.value)
        if product_dd.value  != "ALL":
            sdf = sdf.filter(col("Product_title") == product_dd.value)

        # 2.2 Spark‐side: group by day → count reviews
        daily = (
            sdf
              .withColumn("day",
                  date_trunc("day",
                    from_unixtime(col("timestamp")/1000).cast("timestamp")
                  )
              )
              .groupBy("day")
              .agg(spark_count("*").alias("count"))
              .orderBy("day")
        )

        # 2.3 Bring to Pandas & detect spikes
        pdf = daily.toPandas()
        pdf["day_str"] = pdf["day"].dt.strftime("%Y-%m-%d")
        pdf["roll_avg"] = pdf["count"].rolling(window=7, min_periods=1).mean()
        pdf["spike"]    = pdf["count"] > pdf["roll_avg"] * factor

        dates     = pdf["day_str"].tolist()
        counts    = pdf["count"].tolist()
        line_data = [[d, c] for d, c in zip(dates, counts)]
        spike_pts = [[d, c] for d, c, s in zip(dates, counts, pdf["spike"]) if s]

        # 2.4 Render with ECharts effectScatter
        html = f'''
<div id="chart" style="width:800px; height:400px;"></div>
<script src="https://cdn.jsdelivr.net/npm/echarts@5"></script>
<script src="https://cdn.jsdelivr.net/npm/echarts-liquidfill@3"></script>
<script>
  var chart = echarts.init(document.getElementById('chart'));
  chart.setOption({{
    tooltip: {{ trigger: 'axis' }},
    xAxis: {{
      type: 'category',
      data: {dates},
      axisLabel: {{ rotate: 45 }}
    }},
    yAxis: {{ type: 'value', name: 'Reviews' }},
    series: [
      {{
        name: 'Reviews',
        type: 'line',
        data: {line_data},
        smooth: true,
        lineStyle: {{ color: '#5470C6' }}
      }},
      {{
        name: 'Spikes',
        type: 'effectScatter',
        coordinateSystem: 'cartesian2d',
        data: {spike_pts},
        symbolSize: 12,
        showEffectOn: 'render',
        rippleEffect: {{ brushType: 'stroke', scale: 4 }},
        itemStyle: {{ color: '#EE6666' }}
      }}
    ]
  }});
</script>
'''
        display(HTML(html))

# 3️⃣ Wire callbacks & initial render
for w in (category_dd, product_dd, threshold_sl):
    w.observe(update_timeline, names="value")
update_timeline(None)

Output()

## Review Velocity Spike Alert Dashboard
![Review Velocity Spike Alert Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/review_velocity_spike_alert.png?raw=true)

# Spike Analysis: Key Insights & Strategic Recommendations

## 1. Executive Summary  
We applied a “spike factor” threshold (≥2× local baseline) to daily review counts across five major categories (Baby, Amazon Home, Amazon Fashion, All Beauty, and Overall). This highlights anomalous surges—likely driven by promotions, product launches, seasonal demand, or service issues. Below are the top takeaways and decision-driven recommendations.

---

## 2. Key Observations

### a. Long-Term Growth & Category Maturity  
- **Amazon Fashion** saw explosive early adoption (spikes up to 65 K+ reviews/day around 2017–2018), then plateaued—suggesting market saturation and the need for differentiation in a crowded segment.  
- **Amazon Home** and **All Beauty** exhibit steadier growth curves with moderate spikes (~250–350 reviews/day) tied to mid-year and holiday peaks, reflecting ongoing category expansion.  
- **Baby** reviews remained relatively low-volume (spikes at ~45 reviews/day), but show clear upticks during back-to-school and year-end gifting windows—indicating seasonal buying patterns.  

### b. Spike Frequency & Magnitude  
- **Overall (“ALL”)** aggregate spikes mirror Fashion’s major surges but also capture cross-category promotions (e.g., Prime Day, Black Friday).  
- **Beauty** tight clustering of spikes around summer months and Q4 suggests synchronized marketing campaigns and influencer pushes.  
- **Baby** and **Amazon Home** spikes are more dispersed—offering opportunities to smooth demand with targeted promotions.

### c. Promotional & Seasonal Drivers  
- **Prime Day & Holiday Season** consistently align with the highest-magnitude spikes across all categories—underscoring the impact of platform-wide events.  
- **Category-specific campaigns** (e.g., “Beauty Week,” “Home Essentials Sale”) generate meaningful uplifts but often miss cross-promotional synergy with other categories.

### d. Risk Signals & Customer Experience  
- Sharp negative spikes (dips below baseline) in some periods hint at service interruptions, stock-outs, or sudden product recalls—requiring post-spike root-cause analysis.

---

## 3. Strategic Recommendations

> 1. Validate each spike period against promotional calendars and inventory logs.  
> 2. Conduct root-cause analysis on negative dips to safeguard customer experience.  
> 3. Implement a dashboard prototype that overlays marketing spend, inventory levels, and sentiment metrics on these spikes.  

By translating these anomalous review surges into actionable intelligence, the business can optimize inventory, time promotions for maximum impact, and proactively safeguard the customer experience.  

-----

## 14. Helpfulness vs. Rating

In [ ]:
# 2️⃣ Build Category & Product filters
category_dd = widgets.Dropdown(
    options=["ALL"] + sorted(
        master_df.select("main_category")
                 .where(col("main_category").isNotNull())
                 .distinct()
                 .rdd.map(lambda r: r["main_category"])
                 .collect()
    ),
    description="Category:"
)
product_dd = widgets.Dropdown(
    options=["ALL"],
    description="Product:",
    layout=widgets.Layout(width="300px")
)
def update_products(_):
    cat = category_dd.value
    if cat == "ALL":
        product_dd.options = ["ALL"]
    else:
        titles = (
            master_df.filter(col("main_category")==cat)
                     .select("Product_title")
                     .where(col("Product_title").isNotNull())
                     .distinct()
                     .rdd.map(lambda r: r["Product_title"])
                     .collect()
        )
        product_dd.options = ["ALL"] + sorted(titles)[:200]
category_dd.observe(update_products, names="value")
update_products(None)

display(widgets.HBox([category_dd, product_dd]))
out = widgets.Output()
display(out)

# 3️⃣ Callback: sample, aggregate, and plot
def update_hex_kde(_):
    with out:
        clear_output(wait=True)

        # 3.1) Compute helpfulness_ratio and apply filters
        sdf = master_df.withColumn(
            "helpfulness_ratio",
            col("helpful_vote")/(col("helpful_vote")+1)
        )
        if category_dd.value != "ALL":
            sdf = sdf.filter(col("main_category")==category_dd.value)
        if product_dd.value != "ALL":
            sdf = sdf.filter(col("Product_title")==product_dd.value)

        # 3.2) Sample up to 50 000 rows to avoid OOM
        total = sdf.count()
        frac = min(50000/total, 1.0)
        pdf = (
            sdf.sample(False, frac, seed=42)
               .select("helpfulness_ratio", "rating")
               .toPandas()
        ).dropna()

        if pdf.empty:
            print("No data for this selection.")
            return

        x = pdf["helpfulness_ratio"]
        y = pdf["rating"]

        # 3.3) Center hexbin plot
        bins = hexbin(x, y, 0.05)
        hex_src = ColumnDataSource(dict(q=bins.q, r=bins.r, counts=bins.counts))
        mapper = LinearColorMapper(palette=Viridis256,
                                   low=min(bins.counts),
                                   high=max(bins.counts))

        p_center = figure(
            width=400, height=400,
            x_axis_label="Helpfulness Ratio",
            y_axis_label="Rating",
            tools="pan,box_zoom,reset,hover"
        )
        p_center.hex_tile(
            q="q", r="r", size=0.05,
            source=hex_src,
            fill_color={'field':'counts','transform':mapper},
            line_color=None
        )
        p_center.add_layout(ColorBar(color_mapper=mapper, location=(0,0)), 'right')
        hover = p_center.select_one(HoverTool)
        hover.tooltips = [("Count","@counts")]

        # 3.4) Top marginal KDE for x
        bins_x, edges_x = np.histogram(x, bins=50, density=True)
        centers_x = (edges_x[:-1] + edges_x[1:]) / 2
        kernel_x = np.exp(-0.5*((centers_x-centers_x.mean())/0.1)**2)
        kernel_x /= kernel_x.sum()
        density_x = np.convolve(bins_x, kernel_x, mode='same')

        p_top = figure(
            width=400, height=150,
            x_range=p_center.x_range,
            tools="", toolbar_location=None
        )
        p_top.varea(x=centers_x, y1=0, y2=density_x, alpha=0.6, color="#6baed6")
        p_top.yaxis.visible = False

        # 3.5) Right marginal KDE for y
        bins_y, edges_y = np.histogram(y, bins=50, density=True)
        centers_y = (edges_y[:-1] + edges_y[1:]) / 2
        kernel_y = np.exp(-0.5*((centers_y-centers_y.mean())/0.5)**2)
        kernel_y /= kernel_y.sum()
        density_y = np.convolve(bins_y, kernel_y, mode='same')

        p_right = figure(
            width=150, height=400,
            y_range=p_center.y_range,
            tools="", toolbar_location=None
        )
        p_right.harea(y=centers_y, x1=0, x2=density_y, alpha=0.6, color="#3182bd")
        p_right.xaxis.visible = False

        # 3.6) Layout and show
        layout = gridplot(
            [[p_top,    None],
             [p_center, p_right]],
            merge_tools=False
        )
        show(layout)

# 4️⃣ Wire callbacks & initial draw
category_dd.observe(update_hex_kde, names="value")
product_dd.observe(update_hex_kde, names="value")
update_hex_kde(None)

Output()

## Helpfulness vs Ratings Dashboard
![Helpfulness vs Ratings Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/helpfulness_vs_rating.png?raw=true)

## How to Read the Helpfulness–Rating Composite Chart

| Panel                        | What it Shows                                               | How to Interpret                                                                                                                                               |
|------------------------------|-------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Top-Left**<br>Helpfulness KDE | Smoothed density of each review’s `helpful_votes/total_votes` (0–1) | - **Peak at 0–0.1:** Most reviews get few or no helpful votes.<br>- **Hump at 0.4–0.6:** A minority of reviews are widely deemed helpful.                |
| **Bottom-Left**<br>Hexbin Plot  | **X:** Helpfulness ratio<br>**Y:** Star rating (1–5)<br>**Color:** Number of reviews in each bin | - **Darker bins:** high review volume; **yellow bins:** extreme volume.<br>- **Cluster at (0–0.1, 5★):** Bulk of 5-star reviews rarely voted helpful.<br>- **Off-diagonal points (e.g. 1–2★ & high ratio):** Very helpful negative critiques to spotlight. |
| **Right**<br>Rating KDE        | Vertical density of star ratings (1–5)                      | - **Bulge at 5★:** Strong positive skew.<br>- **Thin tail at 1–2★:** Relatively few negative reviews.                                                          |

---

## Key Insights

- **Low Overall Engagement**  
  ~70–80% of reviews across all star levels have a helpfulness ratio < 0.1—even highly positive reviews go largely unvoted.

- **High-Value Negative Feedback**  
  A small but clear cluster of 1–2★ reviews with high helpfulness ratios represents detailed critiques that users trust.

- **Balanced Mid-Range Reviews**  
  3–4★ reviews form a “shoulder” in the KDE around 0.3–0.5—these balanced pros & cons posts can be highly informative for buyers and product teams.

- **Category Consistency**  
  Every major category (Fashion, Beauty, Devices, Health, etc.) follows the same pattern of heavy 5★ volume, low vote rates, and a niche of high-helpfulness critiques.

---

## 14. Verified-Purchase Impact

In [ ]:
# 1️⃣ Compute average rating by category × verified flag
stats = (
    master_df
      .groupBy("main_category", "verified_purchase")
      .agg(
          spark_count("*").alias("n_reviews"),
          avg(col("rating")).alias("avg_rating")
      )
      .cache()
)

# 2️⃣ Pivot out verified vs non-verified
ver = (
    stats
      .filter(col("verified_purchase") == True)
      .select("main_category", col("avg_rating").alias("avg_verified"))
)
nonver = (
    stats
      .filter(col("verified_purchase") == False)
      .select("main_category", col("avg_rating").alias("avg_nonverified"))
)
df_p = ver.join(nonver, on="main_category", how="inner") \
           .toPandas()

# 3️⃣ Compute delta and sort
df_p["delta"] = df_p["avg_verified"] - df_p["avg_nonverified"]
df_p = df_p.sort_values("delta", ascending=False).reset_index(drop=True)
df_p["y"] = df_p.index  # numerical y-position
categories = df_p["main_category"].tolist()

In [ ]:
category_dd = widgets.Dropdown(
    options=["ALL"] + sorted(
        master_df.select("main_category")
                 .where(col("main_category").isNotNull())
                 .distinct()
                 .rdd.map(lambda r: r["main_category"])
                 .collect()
    ),
    description="Category:"
)
product_dd = widgets.Dropdown(
    options=["ALL"],
    description="Product:",
    layout=widgets.Layout(width="400px")
)

def update_products(_):
    cat = category_dd.value
    if cat == "ALL":
        product_dd.options = ["ALL"]
    else:
        prods = (
            master_df.filter(col("main_category")==cat)
                     .select("Product_title")
                     .where(col("Product_title").isNotNull())
                     .distinct()
                     .rdd.map(lambda r: r["Product_title"])
                     .collect()
        )
        product_dd.options = ["ALL"] + sorted(prods)[:200]

category_dd.observe(update_products, names="value")
update_products(None)

display(widgets.VBox([category_dd, product_dd]))
out = widgets.Output()
display(out)

# 2️⃣ Callback: aggregate and plot
# import pandas as pd
# from bokeh.plotting    import figure, output_notebook, show
# from bokeh.models      import ColumnDataSource, HoverTool
# from bokeh.palettes    import Category10

# output_notebook()

def update_impact(_):
    with out:
        clear_output(wait=True)

        # 2.1 Filter by category & product
        sdf = master_df
        if category_dd.value != "ALL":
            sdf = sdf.filter(col("main_category") == category_dd.value)
        if product_dd.value != "ALL":
            sdf = sdf.filter(col("Product_title") == product_dd.value)

        # 2.2 Choose grouping field
        if product_dd.value != "ALL":
            grp = "Product_title"
            title = f"{category_dd.value} ➤ {product_dd.value}"
        else:
            grp = "main_category"
            title = category_dd.value

        # 2.3 Compute avg ratings by grp × verified flag
        stats = (
            sdf.groupBy(grp, "verified_purchase")
               .agg(
                   spark_count("*").alias("n_reviews"),
                   avg(col("rating")).alias("avg_rating")
               )
               .cache()
        )

        # 2.4 Split verified vs non-verified
        ver = stats.filter(col("verified_purchase") == True) \
                   .select(grp, col("avg_rating").alias("avg_verified"))
        nonver = stats.filter(col("verified_purchase") == False) \
                      .select(grp, col("avg_rating").alias("avg_nonverified"))

        # 2.5 Join and toPandas
        df = ver.join(nonver, on=grp, how="inner") \
                .toPandas()
        if df.empty:
            print("No data for this selection.")
            return

        # 2.6 Compute delta & sort
        df["delta"] = df["avg_verified"] - df["avg_nonverified"]
        df = df.sort_values("delta", ascending=False).reset_index(drop=True)
        df["y"] = df.index
        df["color"] = [Category10[10][i % 10] for i in range(len(df))]

        # 3️⃣ Build Bokeh dumbbell chart
        src = ColumnDataSource(df)
        y_labels = df[grp].tolist()[::-1]

        p = figure(
            y_range=y_labels,
            x_range=(
                min(df["avg_nonverified"].min(), df["avg_verified"].min()) - 0.2,
                max(df["avg_nonverified"].max(), df["avg_verified"].max()) + 0.2
            ),
            width=800,
            height=25 * len(df) + 200,
            title=f"Verified-Purchase Impact: {title}",
            tools="pan,box_zoom,reset,save,hover"
        )
        p.xaxis.axis_label = "Average Rating"
        p.yaxis.axis_label = grp.replace("_", " ").title()

        # 3.1 Draw segments
        p.segment(
            x0="avg_nonverified", y0=grp,
            x1="avg_verified",     y1=grp,
            color="color", line_width=3,
            source=src, legend_field=grp
        )
        # 3.2 Draw circles
        nonv = p.circle(
            x="avg_nonverified", y=grp,
            size=10, fill_color=None, line_color="color",
            source=src, legend_field=grp
        )
        verc = p.circle(
            x="avg_verified", y=grp,
            size=10, fill_color="color", line_color="black",
            source=src, legend_field=grp
        )

        # 3.3 Hover tool
        hover = HoverTool(
            renderers=[nonv, verc],
            tooltips=[
                (grp,        f"@{grp}"),
                ("Non-Ver",  "@avg_nonverified{0.00}"),
                ("Verified", "@avg_verified{0.00}"),
                ("Δ Stars",  "@delta{0.00}")
            ]
        )
        p.add_tools(hover)

        # 3.4 Legend click toggles
        p.legend.click_policy = "hide"

        show(p)

# 4️⃣ Wire callbacks & initial render
category_dd.observe(update_impact, names="value")
product_dd.observe(update_impact, names="value")
update_impact(None)

Output()

## Verified Purchase Impact Dashboard
![Verified Purchase Impact Dashboard](https://github.com/billburr958/images-temp/blob/main/Exploration/verified_purchase_impact.png?raw=true)

# Interpreting the “Verified-Purchase Impact” Chart

- **Y-Axis (Main Category):** Each horizontal line corresponds to one product category.  
- **X-Axis (Average Rating):** Shows mean star rating (1–5) for that category.  
- **Two Markers per Line:**  
  - **Filled Circle:** Average rating **for Verified-Purchase** reviews only.  
  - **Hollow Circle:** Average rating **across all** reviews (verified + unverified).  

A longer horizontal span means a larger gap between “all” and “verified”–only ratings.

---

## Key Insights

1. **Unverified Reviews Inflate Scores**  
   - Across nearly every category, the **hollow marker** (all reviews) sits to the right of the **filled marker** (verified only).  
   - **Software** shows the largest gap (≈3.0 ★ for verified vs. ~5.0 ★ overall), indicating unverified reviewers give near-perfect scores vs. far lower scores from buyers.  

2. **High-Risk Categories for Review Bias**  
   - **Unique Finds, Premium Beauty, Amazon Devices, Car Electronics** and **Tools & Home Improvement** all display 0.5 ★–0.8 ★ differences.  
   - These are prime targets for ***fake-review detection*** and stricter review-gating.

3. **Consistent, Trustworthy Segments**  
   - **Collectible Coins, Video Games, Cell Phones & Accessories** show minimal spread (<0.2 ★), suggesting close alignment between verified and overall ratings.

4. **User Perception vs. Purchasing Reality**  
   - Customers may see overly rosy average scores (all reviews), but actual buyers (verified) report more moderate experiences—potentially driving returns and service tickets.

---

## Strategic Recommendations

1. **Surface Verified-Purchase Metrics Prominently**  
   - In product listings and detail pages, display “Verified Purchase ★4.0” instead of—or alongside—“Overall ★4.6” to set realistic expectations.

2. **Target High-Gap Categories for Trust Initiatives**  
   - Introduce extra verification steps or manual moderation for categories with ≥0.5 ★ gaps (e.g., Software, Premium Beauty).  
   - Leverage machine-learning to flag and remove suspicious unverified reviews in these segments.

3. **Incentivize Genuine Buyer Feedback**  
   - Offer coupons or loyalty points for first-hand purchasers to leave detailed reviews, boosting the volume of verified feedback.

4. **Adjust Search & Recommendation Algorithms**  
   - Weight “verified-only” average ratings more heavily when ranking products—especially in high-bias categories—to promote genuinely well-rated items.

5. **Monitor and Report Gaps Over Time**  
   - Build a simple dashboard tracking the verified vs. overall rating delta by category.  
   - If the gap widens, trigger a review of promotional campaigns, review-solicitation channels, and fraud-detection rules.

By understanding and acting on these verified-purchase vs. all-reviews discrepancies, the business can restore customer trust, reduce returns, and improve long-term sat

-----

In [ ]:
spark.stop()